In [1]:
module RandomSpherePoints

function get_points_spherical_random(n::Int)
    r = ones(n)
    θ = acos.(1-2rand(n))
    ϕ = 2pi*rand(n)
    
    return [r θ ϕ]
end

function get_points_spherical_evenly(n::Int)
    r = ones(n)
    θ = acos.(1-2linspace(0.0, 1.0, n))
    ϕ = 2pi*linspace(0.0, 1.0, n)
    
    return [r θ ϕ]
end

function get_points_cartesian_random(n::Int)
    points = get_points_spherical_random(n)
    
    return [sin.(points[:,2]).*cos.(points[:,3]) sin.(points[:,2]).*sin.(points[:,3]) cos.(points[:,2])]
end

function random_sphere_pack(r::AbstractFloat, R::AbstractFloat, n::Int)
        
    step = r/10
    x = [ collect(-step : -step : -R); collect(0 : step : R)]
    y = [ collect(-step : -step : -R); collect(0 : step : R)]
    z = [ collect(-step : -step : -R); collect(0 : step : R)]
    len = length(x)
    places = zeros(len, len, len)
    
    for i = 1:len
        for j = 1:len
            for k = 1:len
                if sqrt(x[i]^2 + y[j]^2 + z[k]^2) < R
                    places[i,j,k] = 1
                end
            end
        end
    end
    
    points = zeros(n, 3)
    count = 0
    
    while count < n
        vacant_points = find(places)
        if isempty(vacant_points)
            error("there are no more vacant points in the space of the cluster at `count`=$count")
        end
        
        random_vacant_point = vacant_points[rand(1:length(vacant_points))]
        (I,J,K) = ind2sub(places, random_vacant_point)
        
        if places[I,J,K] != 1
            display("Something went wrong! places(p(1),p(2),p(3))!=1")
        end
        
        for i = 1:len
            for j = 1:len
                for k = 1:len
                    if sqrt((x[i]-x[I])^2 + (y[j]-y[J])^2 + (z[k]-z[K])^2) < 2r
                        places[i,j,k] = 0
                    end
                end
            end
        end
        
        count += 1;
        points[count,:] = [x[I] y[J] z[K]]
    end
    
    return points
end

export get_points_spherical_random
export get_points_cartesian_random
export get_points_spherical_evenly
export random_sphere_pack

end

module Usov2016

μ₀=4pi*1e-7

using RandomSpherePoints, Optim, Plots


mutable struct UsovParticle
    n::Int
    νs::Array{Float64,2}
    rs2ds::Array{Float64,2}
    ns::Array{Float64,3}
    ψs::Array{Float64,1}
    ϕs::Array{Float64,1}
    λ::Float64
    hprev::Float64
    hsat::Float64

    function UsovParticle(r::Float64,R::Float64,n::Int,η::Float64,K_M2::Float64)
        ps = random_sphere_pack(r, R, n)
        νs = get_points_cartesian_random(n)
        ϕs = acos.(νs[:,3])
        ψs = atan2.(νs[:,2], νs[:,1])
        rs2ds = zeros(n, n)
        ns = zeros(3, n, n)
        for i = 1:n
            for j = 1:n
                ds = ps[j,:] -  ps[i,:]
                rs2ds[i,j] = (r/norm(ds))^3
                ns[:,i,j] = ds/norm(ds)
            end
        end
        λ = μ₀/(6*K_M2)
        return new(n, νs, rs2ds, ns, ψs, ϕs, λ, 0.0, 4.0)
    end
end

function total_energy(n, νs, ns, λ, ϕs, ψs, rs2ds, h)
    es = [ sin.(ϕs).*cos.(ψs) sin.(ϕs).*sin.(ψs)  cos.(ϕs)]
    
    #sum(es.*nus,2) - scallar multiplication of all particle vectors es and nus 
    energy_anis_and_zeeman = sum( 0.5(1-sum(es.*νs, 2).^2) - h*es[:,3]) #external sum - by particles
    energy_dip_dip = 0
    for i = 1:n
        # external sum by j-th neighbouring particles
        #v1 = λ*rs2ds[i+1:n,i]
        #v21 = (es[i,:]'*ns[:,i+1:n,i])
        #v22 = sum( es[i+1:n,:].*ns[:,i+1:n,i]',2 )
        #v2 = v21'.*v22
        #v3 = es[i+1:n,:]*es[i,:]
        # energy_dip_dip -= sum(v1.*(3v2.-v3))
        energy_dip_dip -= sum(λ*rs2ds[i+1:n,i].* (3*(es[i,:]'*ns[:,i+1:n,i])'.*sum( es[i+1:n,:].*ns[:,i+1:n,i]',2 ) - es[i+1:n,:]*es[i,:]) )
    end
    
    return energy_anis_and_zeeman + energy_dip_dip
end

function total_energy_for_loop(n, νs, ns, λ, ϕs, ψs, rs2ds, h)
    es = @. [ sin(ϕs)*cos(ψs) sin(ϕs)*sin(ψs)  cos(ϕs)]
     
    anisotropy = 0.0
    zeeman = 0.0
    dip_dip = 0.0
    for i = 1:n
        anisotropy += 0.5(1-sum(es[i,:].*νs[i,:]).^2)
        zeeman += h*es[i,3]
        for j = i+1:n
            dip_dip += λ*rs2ds[j,i]* (3*sum(es[i,:].*ns[:,j,i])*sum(es[j,:].*ns[:,j,i]) - sum(es[j,:].*es[i,:]))
        end
    end    
    return anisotropy - zeeman - dip_dip
end


function apply_field_by_step!(p::UsovParticle, h::Float64)
    energy(x) = total_energy(p.n, p.νs, p.ns, p.λ, x[1:p.n], x[p.n+1:2p.n], p.rs2ds, h);
    optimization_result = optimize(energy, [p.ϕs; p.ψs], BFGS());
    angles = Optim.minimizer(optimization_result);
    p.ϕs[1:end] = angles[1:p.n]
    p.ψs[1:end] = angles[p.n+1:2p.n]
    p.hprev = h
end

function apply_field!(p::UsovParticle, applied_field::Float64)
    hstep = 0.01
    if applied_field>p.hprev
        field_values = p.hprev:hstep:applied_field
    else
        field_values = p.hprev:-hstep:applied_field
    end
    
    if field_values[end] != applied_field
        push!(field_values,applied_field)
    end
    
    for h in field_values
        apply_field_by_step!(p, h)
    end
        
end

function create_for_constant_concentration(r::Float64,n::Int,η::Float64,K::Float64,M::Float64)
    R = r*(n/η)^(1/3)
    K_M2 = K/(M*M)
    p = UsovParticle(r, R, n, η, K_M2)
end

function draw_representation(p::UsovParticle)
    hstep = 0.01
    hmax = p.hsat
    h = [collect(hmax:-hstep:-hmax); collect(-hmax:hstep:hmax)]
    len = length(h)
    println(len)
    m = zeros(len)
    
    for i=1:len
        apply_field!(p, h[i])
        m[i] = sum(cos.(p.ϕs))/p.n
    end
    plot(h, m, xlim = (-p.hsat, p.hsat), ylim = (-2,2));
end

function get_magnetization(p::UsovParticle)
    return sum(cos.(p.ϕs))/p.n
end

function magnetize_particle(p::UsovParticle, h::Array{Float64, 1})
    len = length(h)
    
    m = zeros(len)
    for i = 1:len
        apply_field!(p, h[i])
        m[i] = get_magnetization(p)
    end
    
    return m
end

export create_for_constant_concentration, apply_field!, draw_representation, magnetize_particle
end

Usov2016

In [2]:
using Usov2016
r = 20e-9
η = 0.24
n = 10
K = 5e03
M = 800000.0
p = create_for_constant_concentration(r, n, η, K, M)
p.hsat = 12
#draw_representation(p)
p

Usov2016.UsovParticle(10, [0.207392 0.442155 -0.872632; -0.234186 -0.965297 0.115581; … ; -0.0521548 -0.968699 0.242696; 0.30173 -0.949735 0.0834445], [Inf 0.0170595 … 0.00506865 0.0134289; 0.0170595 Inf … 0.00615957 0.014719; … ; 0.00506865 0.00615957 … Inf 0.0534215; 0.0134289 0.014719 … 0.0534215 Inf], [NaN -0.0257428 … -0.910416 -0.736843; NaN 0.566341 … 0.309198 0.665536; NaN 0.823769 … 0.274843 0.118846]

[0.0257428 NaN … -0.953206 -0.735215; -0.566341 NaN … -0.0733236 0.147043; -0.823769 NaN … -0.293294 -0.661693]

[0.254457 0.130455 … -0.903012 -0.893998; -0.933008 0.0 … -0.0768521 0.21456; 0.254457 0.991454 … 0.422686 0.393359]

[0.943701 0.789474 … -0.259259 0.374766; 0.164122 0.526316 … 0.962963 0.899438; -0.287213 0.315789 … 0.0740741 -0.22486]

[0.925758 0.744845 … -0.279776 0.660979; -0.342128 0.0827606 … 0.039968 0.484718; 0.161001 0.662085 … 0.959233 0.572848]

[0.718571 0.893033 … -0.156055 0.646997; -0.630583 -0.390702 … -0.975343 -0.539164; -0.293294 0.223258 … -0.15

In [4]:
using Usov2016, Plots
r = 20e-9
η = 0.24
n = 20
K = 5e03
M = 800000.0
p = create_for_constant_concentration(r, n, η, K, M)

hstep = 0.01
hmax = 6.0
h = [collect(0:hstep:hmax); collect(hmax:-hstep:-hmax); collect(-hmax:hstep:hmax)]
#display(@btime m=calculate_particles(psis, h))
m = magnetize_particle(p, h)
plot(h,m, xlim=(-1.1hmax, 1.1hmax), ylim = (-1.1, 1.1), aspect_ratio=[1 2])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip0502)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 309.908,194.422 310.318,194.381 310.728,194.34 311.138,194.299 311.547,194.258 311.957,194.217 312.367,194.177 312.776,194.136 313.186,194.095 313.596,194.055 
 314.006,194.014 314.415,193.974 314.825,193.933 315.235,193.893 315.644,193.852 316.054,193.812 316.464,193.772 316.874,193.732 317.283,193.692 317.693,193.651 
 318.103,193.611 318.512,193.571 318.922,193.531 319.332,193.491 319.741,193.451 320.151,193.411 320.561,193.371 320.971,193.332 321.38,193.292 321.79,193.252 
 322.2,193.212 322.609,193.172 323.019,193.133 323.429,193.093 323.839,193.053 324.248,193.014 324.658,192.974 325.068,192.934 325.477,192.895 325.887,192.855 
 326.297,192.815 326.706,192.776 327.116,192.736 327.526,192.696 327.936,192.657 328.345,192.617 328.755,192.578 329.165,192.538 329.574,192.499 329.984,192.459 
 330.394,192.419 330.804,192.38 331.213,192.34 331.623,192.301 332.033,192.261 332.442,192.222 332.852,192.182 333.262,192.142 333.671,192.103 334.081,192.063 
 334.491,192.023 334.901,191.984 335.31,191.944 335.72,191.904 336.13,191.865 336.539,191.825 336.949,191.785 337.359,191.745 337.769,191.706 338.178,191.666 
 338.588,191.626 338.998,191.586 339.407,191.546 339.817,191.506 340.227,191.466 340.636,191.426 341.046,191.386 341.456,191.346 341.866,191.306 342.275,191.266 
 342.685,191.226 343.095,191.185 343.504,191.145 343.914,191.105 344.324,191.065 344.734,191.024 345.143,190.984 345.553,190.943 345.963,190.903 346.372,190.862 
 346.782,190.821 347.192,190.781 347.602,190.74 348.011,190.699 348.421,190.658 348.831,190.617 349.24,190.576 349.65,190.535 350.06,190.494 350.469,190.453 
 350.879,190.412 351.289,190.37 351.699,190.329 352.108,190.287 352.518,190.246 352.928,190.204 353.337,190.162 353.747,190.121 354.157,190.079 354.567,190.037 
 354.976,189.995 355.386,189.952 355.796,189.91 356.205,189.868 356.615,189.825 357.025,189.783 357.434,189.74 357.844,189.697 358.254,189.654 358.664,189.611 
 359.073,189.568 359.483,189.525 359.893,189.482 360.302,189.438 360.712,189.394 361.122,189.351 361.532,189.307 361.941,189.263 362.351,189.218 362.761,189.174 
 363.17,189.13 363.58,189.085 363.99,189.04 364.399,188.995 364.809,188.95 365.219,188.904 365.629,188.859 366.038,188.813 366.448,188.767 366.858,188.721 
 367.267,188.675 367.677,188.628 368.087,188.581 368.497,188.534 368.906,188.487 369.316,188.439 369.726,188.391 370.135,188.343 370.545,188.295 370.955,188.246 
 371.365,188.197 371.774,188.148 372.184,188.098 372.594,188.048 373.003,187.998 373.413,187.947 373.823,187.896 374.232,187.844 374.642,187.792 375.052,187.74 
 375.462,187.687 375.871,187.633 376.281,187.579 376.691,187.525 377.1,187.47 377.51,187.414 377.92,187.357 378.33,187.3 378.739,187.242 379.149,187.183 
 379.559,187.124 379.968,187.063 380.378,187.001 380.788,186.938 381.197,186.874 381.607,186.809 382.017,186.742 382.427,186.673 382.836,186.602 383.246,186.529 
 383.656,186.454 384.065,186.376 384.475,186.295 384.885,186.21 385.295,186.12 385.704,186.024 386.114,185.919 386.524,185.804 386.933,185.67 387.343,185.503 
 387.753,185.176 388.162,181.045 388.572,181.006 388.982,180.967 389.392,180.928 389.801,180.889 390.211,180.85 390.621,180.811 391.03,180.773 391.44,180.734 
 391.85,180.696 392.26,180.657 392.669,180.619 393.079,180.581 393.489,180.543 393.898,180.505 394.308,180.468 394.718,180.43 395.127,180.393 395.537,180.355 
 395.947,180.318 396.357,180.281 396.766,180.244 397.176,180.207 397.586,180.17 397.995,180.133 398.405,180.096 398.815,180.06 399.225,180.023 399.634,179.987 
 400.044,179.951 400.454,179.914 400.863,179.878 401.273,179.842 401.683,179.806 402.093,179.771 402.502,179.735 402.912,179.699 403.322,179.6

In [5]:
plot(h,m, xlim=(-1.1hmax, 1.1hmax), ylim = (-1.1, 1.1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip0702)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 309.908,196.191 310.318,196.025 310.728,195.86 311.138,195.694 311.547,195.529 311.957,195.365 312.367,195.2 312.776,195.036 313.186,194.871 313.596,194.708 
 314.006,194.544 314.415,194.38 314.825,194.217 315.235,194.054 315.644,193.891 316.054,193.728 316.464,193.566 316.874,193.403 317.283,193.241 317.693,193.079 
 318.103,192.917 318.512,192.755 318.922,192.593 319.332,192.432 319.741,192.27 320.151,192.109 320.561,191.948 320.971,191.787 321.38,191.626 321.79,191.465 
 322.2,191.305 322.609,191.144 323.019,190.983 323.429,190.823 323.839,190.663 324.248,190.502 324.658,190.342 325.068,190.182 325.477,190.022 325.887,189.862 
 326.297,189.702 326.706,189.542 327.116,189.382 327.526,189.222 327.936,189.062 328.345,188.902 328.755,188.742 329.165,188.582 329.574,188.422 329.984,188.263 
 330.394,188.103 330.804,187.943 331.213,187.783 331.623,187.623 332.033,187.463 332.442,187.303 332.852,187.143 333.262,186.983 333.671,186.823 334.081,186.663 
 334.491,186.503 334.901,186.343 335.31,186.183 335.72,186.022 336.13,185.862 336.539,185.701 336.949,185.541 337.359,185.38 337.769,185.219 338.178,185.059 
 338.588,184.898 338.998,184.737 339.407,184.575 339.817,184.414 340.227,184.253 340.636,184.091 341.046,183.929 341.456,183.768 341.866,183.606 342.275,183.443 
 342.685,183.281 343.095,183.119 343.504,182.956 343.914,182.793 344.324,182.63 344.734,182.467 345.143,182.304 345.553,182.14 345.963,181.977 346.372,181.813 
 346.782,181.648 347.192,181.484 347.602,181.319 348.011,181.155 348.421,180.989 348.831,180.824 349.24,180.659 349.65,180.493 350.06,180.327 350.469,180.16 
 350.879,179.993 351.289,179.826 351.699,179.659 352.108,179.492 352.518,179.324 352.928,179.155 353.337,178.987 353.747,178.818 354.157,178.649 354.567,178.479 
 354.976,178.309 355.386,178.139 355.796,177.968 356.205,177.797 356.615,177.625 357.025,177.453 357.434,177.281 357.844,177.108 358.254,176.935 358.664,176.761 
 359.073,176.587 359.483,176.412 359.893,176.237 360.302,176.061 360.712,175.885 361.122,175.708 361.532,175.53 361.941,175.353 362.351,175.174 362.761,174.995 
 363.17,174.815 363.58,174.634 363.99,174.453 364.399,174.271 364.809,174.089 365.219,173.906 365.629,173.722 366.038,173.537 366.448,173.351 366.858,173.165 
 367.267,172.977 367.677,172.789 368.087,172.6 368.497,172.41 368.906,172.219 369.316,172.027 369.726,171.833 370.135,171.639 370.545,171.444 370.955,171.247 
 371.365,171.049 371.774,170.85 372.184,170.649 372.594,170.447 373.003,170.244 373.413,170.039 373.823,169.832 374.232,169.624 374.642,169.414 375.052,169.202 
 375.462,168.988 375.871,168.772 376.281,168.554 376.691,168.334 377.1,168.111 377.51,167.885 377.92,167.657 378.33,167.426 378.739,167.192 379.149,166.954 
 379.559,166.713 379.968,166.468 380.378,166.218 380.788,165.964 381.197,165.705 381.607,165.44 382.017,165.17 382.427,164.892 382.836,164.607 383.246,164.313 
 383.656,164.01 384.065,163.695 384.475,163.366 384.885,163.022 385.295,162.658 385.704,162.27 386.114,161.849 386.524,161.382 386.933,160.843 387.343,160.166 
 387.753,158.845 388.162,142.163 388.572,142.004 388.982,141.846 389.392,141.688 389.801,141.53 390.211,141.373 390.621,141.217 391.03,141.061 391.44,140.905 
 391.85,140.75 392.26,140.596 392.669,140.442 393.079,140.288 393.489,140.135 393.898,139.982 394.308,139.83 394.718,139.678 395.127,139.526 395.537,139.375 
 395.947,139.225 396.357,139.074 396.766,138.925 397.176,138.775 397.586,138.626 397.995,138.478 398.405,138.33 398.815,138.182 399.225,138.035 399.634,137.888 
 400.044,137.741 400.454,137.595 400.863,137.449 401.273,137.304 401.683,137.159 402.093,137.014 402.502,136.87 402.912,136.726 403.322,136.582